In [3]:
using Random
using JuMP
using Cbc
using IterTools

In [12]:
B = 8
n = 4
S = Int[1, 3, 6, 7]
I = collect(1:n)

4-element Array{Int64,1}:
 1
 2
 3
 4

In [86]:
for ss in subsets(I)
    length(ss) == 0 && continue
    @show ss
end

ss = [1]
ss = [2]
ss = [1, 2]
ss = [3]
ss = [1, 3]
ss = [2, 3]
ss = [1, 2, 3]
ss = [4]
ss = [1, 4]
ss = [2, 4]
ss = [1, 2, 4]
ss = [3, 4]
ss = [1, 3, 4]
ss = [2, 3, 4]
ss = [1, 2, 3, 4]


In [35]:
J = []
for s in subsets(I)
    length(s) == 0 && continue
    Ws = sum(S[i] for i in s)
    Ws > B && continue
    println(s, " ", Ws)
    push!(J, s)
end
sort!(J, lt=(x, y) -> length(x) < length(y))
JJ = collect(1:length(J))

[1] 1
[2] 3
[1, 2] 4
[3] 6
[1, 3] 7
[4] 7
[1, 4] 8


7-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6
 7

In [58]:
di = Dict{Int,Set{Int}}(key => Set{Int}() for key in JJ)
for (idx, s) in enumerate(J)
    println(idx, " ", s)
    for j in s
        push!(di[j], idx)
    end
end
di

1 [1]
2 [2]
3 [3]
4 [4]
5 [1, 2]
6 [1, 3]
7 [1, 4]


Dict{Int64,Set{Int64}} with 7 entries:
  7 => Set(Int64[])
  4 => Set([7, 4])
  2 => Set([2, 5])
  3 => Set([3, 6])
  5 => Set(Int64[])
  6 => Set(Int64[])
  1 => Set([7, 5, 6, 1])

In [66]:
model = Model(with_optimizer(Cbc.Optimizer))

@variable(model, x[j=1:length(J)], Bin)

for (key, value) in di
    length(value) == 0 && continue
    xkey = AffExpr()
    for v in value
        xkey += x[v]
    end
    @constraint(model, xkey ≥ 1)
end


@objective(model, Min, sum(x[:]))

model

A JuMP Model
Minimization problem with:
Variables: 7
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 7 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: x

In [67]:
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3 - 0.00 seconds
Cgl0004I processed model has 4 rows, 7 columns (7 integer (7 of which binary)) and 10 elements
Cbc0012I Integer solution of 3 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 3, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 3 to 3
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after adding r

In [69]:
for i in JJ
    println(i, ":", value(x[i]))
end

1:0.0
2:1.0
3:1.0
4:0.0
5:0.0
6:0.0
7:1.0


## 緩和問題

In [76]:
model = Model(with_optimizer(Cbc.Optimizer))

@variable(model, x[j=1:length(J)] ≥ 0)

for (key, value) in di
    length(value) == 0 && continue
    xkey = AffExpr()
    for v in value
        xkey += x[v]
    end
    @constraint(model, xkey ≥ 1)
end


@objective(model, Min, sum(x[:]))
model

A JuMP Model
Minimization problem with:
Variables: 7
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 7 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: x

In [77]:
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 4 (0) rows, 7 (0) columns and 10 (0) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 7.9234384e-06 ( 0.00079234384%) - largest zero change 0
0  Obj 0 Primal inf 3.9999996 (4)
4  Obj 3.0000069
Optimal - objective value 3
Optimal objective 3 - 4 iterations time 0.002
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [78]:
objective_value(model)

3.0

## 緩和問題 (1-3)

In [81]:
model = Model(with_optimizer(Cbc.Optimizer))

@variable(model, 1 >= x[j=1:length(J)] >= 0)

for (key, value) in di
    length(value) == 0 && continue
    xkey = AffExpr()
    for v in value
        xkey += x[v]
    end
    @constraint(model, xkey ≥ 1)
end


@objective(model, Min, sum(x[:]))
model

A JuMP Model
Minimization problem with:
Variables: 7
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 7 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 7 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: x

In [82]:
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 4 (0) rows, 7 (0) columns and 10 (0) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 7.9234384e-06 ( 0.00079234384%) - largest zero change 0
0  Obj 0 Primal inf 3.9999996 (4)
4  Obj 3.0000069
Optimal - objective value 3
Optimal objective 3 - 4 iterations time 0.002
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [83]:
objective_value(model)

3.0